As the number of groups increases, which algorithm performs better at scaling?

In [6]:
import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Step 1: Load the data
df = pd.read_parquet(
    "results/psc-baselines-2.parquet", 
    columns=["algorithm", "n_groups", "np", "value"],
    filters=[("dist", "==", "uniform"), ("n_rows", "==", 8_000_000), ("attribute", "==", "aggregation_time")],
)

# Step 2: Preprocess
df["value"] = df["value"].apply(lambda x: float(x.removesuffix("ms")))
df = df.groupby(["algorithm", "n_groups", "np"])["value"].mean().reset_index()
df = df.rename(columns={"value": "latency"})

# Step 3: Add Speedup Column
baseline = df[df["np"] == 1][["algorithm", "n_groups", "latency"]]
baseline = baseline.rename(columns={"latency": "baseline_latency"})
df = df.merge(baseline, on=["algorithm", "n_groups"], how="left")
df["speedup"] = df["baseline_latency"] / df["latency"]
df = df.drop(columns=["baseline_latency"])

# Step 4: Add Tolerance Column
# Find the latency at the minimal n_groups for each (dist, algorithm, np)
min_group_latency = (
    df.loc[df.groupby(["algorithm", "np"])["n_groups"].idxmin()]
    [["algorithm", "np", "latency"]]
    .rename(columns={"latency": "min_group_latency"})
)

df = df.merge(min_group_latency, on=["algorithm", "np"], how="left")
df["slowdown"] = df["latency"] / df["min_group_latency"]
df = df.drop(columns=["min_group_latency"])

# Step 5: Display
for algorithm in df["algorithm"].unique():
    display(df[(df["np"] == df["np"].max()) & (df["algorithm"] == algorithm)])


,algorithm,n_groups,np,latency,speedup,slowdown
7,duckdbish-two-phase,2000,128,21.0,8.352381,1.000000
15,duckdbish-two-phase,20000,128,33.2,5.451807,1.580952
23,duckdbish-two-phase,200000,128,91.2,4.192982,4.342857
31,duckdbish-two-phase,2000000,128,104.0,8.559615,4.952381


,algorithm,n_groups,np,latency,speedup,slowdown
39,implicit-repartitioning,2000,128,81.6,1.017157,1.000000
47,implicit-repartitioning,20000,128,78.8,1.441624,0.965686
55,implicit-repartitioning,200000,128,80.2,3.638404,0.982843
63,implicit-repartitioning,2000000,128,59.0,15.274576,0.723039


,algorithm,n_groups,np,latency,speedup,slowdown
71,lock-free-hash-table,2000,128,63.0,2.457143,1.000000
79,lock-free-hash-table,20000,128,60.0,2.653333,0.952381
87,lock-free-hash-table,200000,128,54.0,3.000000,0.857143
95,lock-free-hash-table,2000000,128,77.6,4.518041,1.231746


,algorithm,n_groups,np,latency,speedup,slowdown
103,two-phase-central-merge-xxhash,2000,128,24.4,3.278689,1.000000
111,two-phase-central-merge-xxhash,20000,128,50.0,2.444000,2.049180
119,two-phase-central-merge-xxhash,200000,128,350.0,0.878857,14.344262
127,two-phase-central-merge-xxhash,2000000,128,1351.2,0.687981,55.377049


,algorithm,n_groups,np,latency,speedup,slowdown
135,two-phase-radix-xxhash,2000,128,6.0,29.466667,1.000000
143,two-phase-radix-xxhash,20000,128,21.4,8.747664,3.566667
151,two-phase-radix-xxhash,200000,128,35.4,11.372881,5.900000
159,two-phase-radix-xxhash,2000000,128,49.6,17.677419,8.266667
